In [19]:
import pandas as pd
import sqlite3
from tqdm import tqdm
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [20]:
stock_conn = sqlite3.connect('C:/Users/Pranay/Desktop/Work_Folder/Folders/Quality_Equity/OHLC_data.db')
stock_cursor = stock_conn.cursor()
stock_cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = stock_cursor.fetchall()

In [21]:
cf1 = pd.DataFrame(columns = ['Symbol','pct_fall'])
cf2 = pd.DataFrame(columns = ['Symbol','pct_rise'])
for i in tqdm(tables) :
    Stock_symbol = i[0]
    df1 = pd.read_sql_query(f"SELECT * FROM {Stock_symbol}", stock_conn)
    df1['Date'] = [datetime.strptime(date_string,'%Y-%m-%d') for date_string in df1['Date']]
    
    index_52_week = df1['Date'].searchsorted(df1['Date'].iloc[-1] - relativedelta(years=1))

    High_52_week = df1['High'].iloc[index_52_week:].max()
    Low_52_week = df1['Low'].iloc[index_52_week:].min()

    Trading_days_since_52weekHigh = df1.tail(1).index[0] - df1[['High']].iloc[index_52_week:].idxmax()[0]
    Trading_days_since_52weekLow = df1.tail(1).index[0] - df1[['Low']].iloc[index_52_week:].idxmin()[0]
    
    if Trading_days_since_52weekHigh > Trading_days_since_52weekLow :
        pct_fall = (Low_52_week/High_52_week-1)*100
        row_dict = {'Symbol' : Stock_symbol, 'pct_fall' : pct_fall }
        row_df = pd.DataFrame(row_dict,index = [0])
        cf1 = pd.concat([cf1,row_df])
    
    elif Trading_days_since_52weekLow > Trading_days_since_52weekHigh :
        pct_rise = (High_52_week/Low_52_week-1)*100
        row_dict = {'Symbol' : Stock_symbol, 'pct_rise' : pct_rise }
        row_df = pd.DataFrame(row_dict,index = [0])
        cf2 = pd.concat([cf2,row_df])
        
cf1.reset_index(drop = True,inplace = True) 
cf2.reset_index(drop = True,inplace = True)

100%|██████████████████████████████████████████████████████████████████████████████| 1866/1866 [01:46<00:00, 17.58it/s]


In [22]:
cf1.head()

,Symbol,pct_fall
0,nse_21STCENMGM,-46.984127
1,nse_3IINFOLTD,-56.164384
2,nse_5PAISA,-34.443069
3,nse_63MOONS,-56.600278
4,nse_A2ZINFRA,-58.823529


In [23]:
cf4 = cf1.sort_values(by = 'pct_fall')

In [24]:
cf4.head(15)

,Symbol,pct_fall
570,nse_ORIENTCEM,-99.189487
434,nse_KOTHARIPRO,-99.172283
273,nse_GODHA,-95.630252
243,nse_FRETAIL,-94.711538
241,nse_FLFL,-92.024014
236,nse_FEL,-90.909091
234,nse_FCONSUMER,-90.131579
147,nse_CEREBRAINT,-89.902081
442,nse_KSHITIJPOL,-89.452710
677,nse_SELMC,-88.612208


In [25]:
cf2.head()

,Symbol,pct_rise
0,nse_20MICRONS,98.342999
1,nse_360ONE,58.601715
2,nse_3MINDIA,45.944538
3,nse_3PLAND,181.923077
4,nse_4THDIM,704.166667


In [26]:
cf3 = cf2.sort_values(by = 'pct_rise', ascending = False)

In [27]:
cf3.head(15)

,Symbol,pct_rise
251,nse_EMAMIREAL,11787.511916
703,nse_RAJRILTD,6048.148148
719,nse_REGENCERAM,2205.263158
142,nse_BSHSL,2039.826422
480,nse_KOHINOOR,1643.243243
648,nse_PARASPETRO,1450.000000
829,nse_SPCENET,1271.739130
986,nse_ZENITHSTL,1109.523810
381,nse_IMPEXFERRO,1088.888889
463,nse_KEEPLEARN,1008.333333
